___
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://samyzaf.com/ML/cifar10/cifar1.jpg" width="380px" height="180px" />


# <font color= #bbc28d> **Trainning CNN'S - CIFAR Data Augmentation** </font>
#### <font color= #2E9AFE> `Lab 2 - Machine Learning`</font>
- <Strong> Sofía Maldonado, Diana Valdivia, Samantha Sánchez, Isabel Valladolid & Vivienne Toledo </Strong>
- <Strong> Fecha </Strong>: 27/10/2025.

___

<p style="text-align:right;"> Image retrieved from: https://samyzaf.com/ML/cifar10/cifar1.jpg</p>

# <font color=#bbc28d>**Abstract**</font>
This project focuses on building a **custom dataset** for an **image classification problem** using the **CIFAR-10** collection, which includes **10 distinct classes** of images.

# <font color=#bbc28d>**Configuration**</font>
Basic configurations to make the process reproducible and also faster using CUDA if possible:

In [ ]:
# General Libraries
import numpy as np
import random

# Modeling
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
# Utilizar la GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [8]:
#Fijar un a semilla para reproducibilidad
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# <font color=#bbc28d>**Data Augmentation**</font>
- **Selection:** 60 original images were chosen per class, ensuring a total of **600 base samples**.  
- **Data Augmentation:** Each image was transformed using techniques such as  
  - *Random horizontal flips*  
  - *Rotations (±15°)*  
  - *Color jittering* (brightness, contrast, saturation)  
  - *Random resized cropping*  
- **Result:** 10 augmented variants were generated for each image, producing **6,000 additional samples**, for a total of **6,600 images**.

All images and labels were converted into **PyTorch tensors** and stored in a `.pt` file for reusability.  

In [ ]:
# Convertir a tensor las imágenes originales
to_tensor = transforms.ToTensor()

# Data Augmentation: aplica variaciones aleatorias
augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),        # voltea horizontalmente
    transforms.RandomRotation(15),            # rota ±15 grados
    transforms.ColorJitter(0.2, 0.2, 0.2),    # cambia brillo, contraste y saturación
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),  # recorta aleatoriamente
    transforms.ToTensor()
 ])

# Descargar el dataset de Cifar-10
base = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=to_tensor)

# Seleccioanr del dataset original solo 60 imágenes por clase [Total=600]
idxs = []
for c in range(10):
    count = 0
    for i, (_, label) in enumerate(base):
        if label == c:
            idxs.append(i)
            count += 1
            if count == 60:
                break

subset_imgs = []
subset_labels = []

for i in idxs:
    img, lbl = base[i]
    subset_imgs.append(img)
    subset_labels.append(lbl)

# Con Data Augmentation generar 10 variantes por imagen [Total=600x10=6000]
augmented_imgs = []
augmented_labels = []

for i, img in enumerate(subset_imgs):
    lbl = subset_labels[i]
    for j in range(10):
        seed = SEED + i * 1000 + j
        random.seed(seed)
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        img_aug = augment(transforms.ToPILImage()(img))
        augmented_imgs.append(img_aug)
        augmented_labels.append(lbl)

# convierte las listas de imágenes y etiquetas a tensores de PyTorch
# Originales
subset_tensor = torch.stack(subset_imgs)         # [600, 3, 32, 32]
subset_labels_tensor = torch.tensor(subset_labels)  # [600]

# Aumentadas
aug_tensor = torch.stack(augmented_imgs)        # [6000, 3, 32, 32]
aug_labels_tensor = torch.tensor(augmented_labels)  # [6000]

# Juntar las originales + aumentadas
all_imgs_tensor = torch.cat([subset_tensor, aug_tensor], dim=0)   # [6600, 3, 32, 32]
all_labels_tensor = torch.cat([subset_labels_tensor, aug_labels_tensor], dim=0)  # [6600]

# Guardar en un archivo para no tener que descargar todo el dataset cifar10
torch.save((all_imgs_tensor, all_labels_tensor), r"C:\Users\denis\Downloads\cifar10_subset.pt")
print("Se guardaron correctamente las imagenes")

Se guardaron correctamente las imagenes


This section loads the previously saved CIFAR-10 tensors and prepares them for model training. The dataset is created using `TensorDataset`, then divided into training (80%) and testing (20%) sets using a random split with a fixed seed to ensure reproducibility. Data loaders are built for both sets, allowing the data to be processed in batches of 512 images during training and evaluation. Finally, a sample batch is retrieved to confirm the dimensions of the images and labels.

In [9]:
# Cargar los tensores guardados en el archivo
all_imgs_tensor, all_labels_tensor = torch.load(r"C:\Users\denis\Downloads\cifar10_subset.pt")
print("Total imágenes:", all_imgs_tensor.shape[0])

# Crear el dataset
dataset = TensorDataset(all_imgs_tensor, all_labels_tensor)

# Train/Test Split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Random split para poder poner semilla
train_set, test_set = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(SEED))

# Crear los dataloaders para hacerlos por batches
train_loader = DataLoader(train_set, batch_size=512, shuffle=True)
test_loader = DataLoader(test_set, batch_size=512, shuffle=False)

# Verificar dimensiones y particiones
imgs, labels = next(iter(train_loader))
print("Batch de imágenes:", imgs.shape)
print("Batch de etiquetas:", labels.shape)

Total imágenes: 6600
Batch de imágenes: torch.Size([512, 3, 32, 32])
Batch de etiquetas: torch.Size([512])
